# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

1. [ПОДГОТОВКА ДАННЫХ](#shag_1)

2. [ИССЛЕДОВАНИЕ ЗАДАЧИ](#shag_2)

3. [БОРЬБА С ДИСБАЛАНСОМ](#shag_3)

4. [ТЕСТИРОВАНИЕ МОДЕЛИ](#shag_4)


<a id='shag_1'></a>

# 1. Подготовка данных

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.
Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.
Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.
Инструкция по выполнению проекта
Загрузите и подготовьте данные. Поясните порядок действий.
Исследуйте баланс классов, обучите модель без учёта дисбаланса. Кратко опишите выводы.
Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую. Кратко опишите выводы.
Проведите финальное тестирование.

Как вы готовите данные к обучению? Все ли типы признаков обрабатываете?
Хорошо ли поясняете этапы предобработки?
Как исследуете баланс классов?
Изучаете ли модель без учёта дисбаланса классов?
Какие выводы об исследовании задачи делаете?
Корректно ли разбиваете данные на выборки?
Как работаете с несбалансированными классами?
Правильно ли проводите обучение, валидацию и финальное тестирование модели?
Насколько высокое значение F1-меры получаете?
Изучаете ли значения метрики AUC-ROC?
Следите за структурой проекта и поддерживаете аккуратность кода?

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import datetime
#import plotly.graph_objs as go
from scipy import stats as st
from scipy.stats import bartlett
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, accuracy_score
import random
from sklearn.datasets import load_breast_cancer
import warnings

In [5]:
data = pd.read_csv('/datasets/Churn.csv')
display(data)
print(data.info())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


Номер строки, айди и имя не влияют на результат. Избавимся от этих столбцов. Также видим, что Tenure имеет пропущенные значения.

In [6]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
#data.Tenure.fillna(0, inplace=True)

In [7]:
#plt.matshow(data.corr())
#plt.show()
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CreditScore,1,-0.004,-6.2e-05,0.0063,0.012,-0.0055,0.026,-0.0014,-0.027
Age,-0.004,1,-0.013,0.028,-0.031,-0.012,0.085,-0.0072,0.29
Tenure,-6.2e-05,-0.013,1,-0.0079,0.012,0.027,-0.032,0.011,-0.017
Balance,0.0063,0.028,-0.0079,1,-0.3,-0.015,-0.01,0.013,0.12
NumOfProducts,0.012,-0.031,0.012,-0.3,1,0.0032,0.0096,0.014,-0.048
HasCrCard,-0.0055,-0.012,0.027,-0.015,0.0032,1,-0.012,-0.0099,-0.0071
IsActiveMember,0.026,0.085,-0.032,-0.01,0.0096,-0.012,1,-0.011,-0.16
EstimatedSalary,-0.0014,-0.0072,0.011,0.013,0.014,-0.0099,-0.011,1,0.012
Exited,-0.027,0.29,-0.017,0.12,-0.048,-0.0071,-0.16,0.012,1


<div class="alert alert-info">  До заполнения пропущенных значений проверили, с чем коррелирует признак Tenure. Есть слабая корреляция с балансом. Заполним пропущенные значения Tenure случайным значением. </div>

In [8]:
data.Tenure.fillna(random.randint(0,10), inplace=True)  
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB
None


Есть категориальные признаки: пол и страна. 

In [9]:
features = data.drop('Exited',1)
target = data['Exited']
features = pd.get_dummies(features, drop_first=True)

Разделим данные на выборки и проверим размеры выборок.

In [10]:
features_t, features_valid, target_t, target_valid = train_test_split(
    features, target, test_size=0.20, random_state=12345) 

features_train, features_test, target_train, target_test = train_test_split(
    features_t, target_t, test_size=0.25, random_state=12345) 
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_train.shape)
print(target_train.shape)

(2000, 11)
(2000,)
(2000, 11)
(2000,)
(6000, 11)
(6000,)


In [11]:
data.groupby(['Gender', 'Geography']).size().reset_index(name='counts')

,Gender,Geography,counts
0,Female,France,2261
1,Female,Germany,1193
2,Female,Spain,1089
3,Male,France,2753
4,Male,Germany,1316
5,Male,Spain,1388


Количество значений по признакам достаточное, чтобы не было перевеса на один из признаков.

<a id='shag_2'></a>

# 2. Исследование задачи

Для повторяющегося кода введем функцию

In [12]:
def DecisionTreeClassifier_class_weight(features, target, class_w):
    best_score = 0
    best_depth = 0
    best_auc_roc = 0
    for depth in range(1, 16, 1):
        model = DecisionTreeClassifier(class_weight=class_w, max_depth=depth, random_state=12345).fit(features, target)
        f1 = f1_score(target_valid, model.predict(features_valid))
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid, average=None)
        if f1 > best_score:
            best_score = f1
            best_depth = depth
            best_auc_roc = auc_roc
    print('Лучший f1_score: ', best_score,'Лучшая глубина: ', best_depth, 'best_auc_roc', best_auc_roc)

In [29]:
def RandomForestClassifier_build(features, target):
    best_score = 0
    best_depth = 0
    best_auc_roc = 0
    best_n_est = 0
    for depth in range(1, 16, 1):
        for n_est in range(10, 150, 10):
            model = RandomForestClassifier(max_depth=depth, n_estimators=n_est, random_state=12345).fit(features, target)
            f1 = f1_score(target_test, model.predict(features_valid))
            probabilities_valid = model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc = roc_auc_score(target_test, probabilities_one_valid, average=None)
            if f1 > best_score:
                best_score = f1
                best_depth = depth
                best_auc_roc = auc_roc
                best_n_est = n_est
    print('Лучший f1_score: ', best_score,'Лучшая глубина: ', best_depth, 'best_auc_roc', best_auc_roc, 
      'best_n_estimators', best_n_est)

In [19]:
DecisionTreeClassifier_class_weight(features_train, target_train, None)
warnings.filterwarnings('ignore')

Лучший f1_score:  0.5608011444921316 Лучшая глубина:  5 best_auc_roc 0.8468610376210971


В нашей модели значение F1 не достигает необходимого по условию 0.59. Проверим также другую модель на данной выборке.

In [20]:
def LogisticRegression_class_weight(features, target):
    best_score = 0
    best_depth = 0
    best_auc_roc = 0
    for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
        model = LogisticRegression(solver=solver, random_state=12345).fit(features, target)
        f1 = f1_score(target_valid, model.predict(features_valid))
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid, average=None)
        if f1 > best_score:
            best_solver = solver
            best_score = f1
            best_auc_roc = auc_roc
    print('Лучший f1_score: ', best_score,'Лучшая solver: ', solver, 'best_auc_roc', best_auc_roc)
LogisticRegression_class_weight(features_train, target_train)

Лучший f1_score:  0.2490974729241877 Лучшая solver:  saga best_auc_roc 0.7514393207388735


В данном случае f1_score еще ниже = 0.28.

<div class="alert alert-info"> Попробуем модель случайного леса </div>

In [21]:
RandomForestClassifier_build(features_train, target_train)

Лучший f1_score:  0.19005847953216376 Лучшая глубина:  13 best_auc_roc 0.5036813385067054 best_n_estimators 5


In [22]:
print(target[target == 0].count())
print(target[target == 1].count())

7963
2037


Видим, что значений класса 1 примерно в 4 раза меньше, чем значений класса 0.

<a id='shag_3'></a>

# 3. Борьба с дисбалансом

Попробуем модель DecisionTreeClassifier со значением параметра class_weight = 'balanced':

In [23]:
DecisionTreeClassifier_class_weight(features_train, target_train, 'balanced')

Лучший f1_score:  0.601593625498008 Лучшая глубина:  8 best_auc_roc 0.8089645079212888


Теперь f1_score = 0.60 и удовлетворяет условию. При этом значение  auc_roc стало 0.80, вместо 0.84.

Увеличим выборку с классом 1 в 4 раза:

In [24]:
#увеличение выборки
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [25]:
DecisionTreeClassifier_class_weight(features_upsampled, target_upsampled, None)

Лучший f1_score:  0.5994005994005994 Лучшая глубина:  8 best_auc_roc 0.8064923154341932


Результат после ручного увеличения малочисленного класса получился аналогичным модели в параметром class_weight = 'balanced'.

In [26]:
LogisticRegression_class_weight(features_upsampled, target_upsampled)

Лучший f1_score:  0.5041322314049587 Лучшая solver:  saga best_auc_roc 0.7638605805520857


Модель LogisticRegression после ручного увеличения малочисленного класса стала значительно эффективнее. Но пока F1 ниже необходимого.

In [27]:
RandomForestClassifier_build(features_upsampled, target_upsampled)

Лучший f1_score:  0.2897350993377484 Лучшая глубина:  1 best_auc_roc 0.5119269963234301 best_n_estimators 45


Попробуем добиться баланса через уменьшение выборки

In [30]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.32)

Посмотрим какого значения F1 и AUC-ROC можем добиться в зависимости от фракции

In [31]:
for fraction in range(20, 40, 1):
    features_downsampled, target_downsampled = downsample(features_train, target_train, fraction/100)
    print(fraction/100)
    DecisionTreeClassifier_class_weight(features_downsampled, target_downsampled, None)

0.2
Лучший f1_score:  0.5603944124897289 Лучшая глубина:  5 best_auc_roc 0.8105076741440378
0.21
Лучший f1_score:  0.5729076790336496 Лучшая глубина:  4 best_auc_roc 0.8300514686505744
0.22
Лучший f1_score:  0.579372197309417 Лучшая глубина:  6 best_auc_roc 0.8239197464234721
0.23
Лучший f1_score:  0.5657781599312123 Лучшая глубина:  6 best_auc_roc 0.8124550561212259
0.24
Лучший f1_score:  0.5790934320074006 Лучшая глубина:  6 best_auc_roc 0.8292489924382622
0.25
Лучший f1_score:  0.5839285714285714 Лучшая глубина:  7 best_auc_roc 0.818658986319195
0.26
Лучший f1_score:  0.6016096579476862 Лучшая глубина:  7 best_auc_roc 0.821499662781332
0.27
Лучший f1_score:  0.5962962962962963 Лучшая глубина:  6 best_auc_roc 0.8410799334793373
0.28
Лучший f1_score:  0.6033630069238379 Лучшая глубина:  7 best_auc_roc 0.824649270252847
0.29
Лучший f1_score:  0.6161825726141079 Лучшая глубина:  7 best_auc_roc 0.835289896392728
0.3
Лучший f1_score:  0.6115035317860747 Лучшая глубина:  7 best_auc_roc 0.8

Итак, хороший результат получаем при фракции 0.32

In [32]:
DecisionTreeClassifier_class_weight(features_downsampled, target_downsampled, None)

Лучший f1_score:  0.6245919477693145 Лучшая глубина:  8 best_auc_roc 0.8150887860276831


Со значением фракции 0.25 значение F1 было бы близким к значению F1, полученным при upsample. Но благодаря оптимизации фракции, удалось добиться значения F1 0.62.

<div class="alert alert-info">  Проверим также модель случайного леса на сбалансированной выборке. </div>

In [34]:
RandomForestClassifier_build(features_downsampled, target_downsampled)

Лучший f1_score:  0.25418060200668896 Лучшая глубина:  11 best_auc_roc 0.5276998469288003 best_n_estimators 10


In [35]:
LogisticRegression_class_weight(features_downsampled, target_downsampled)

Лучший f1_score:  0.5015673981191222 Лучшая solver:  saga best_auc_roc 0.764503752581249


На примере модели LogisticRegression убеждаемся, что действительно upsample и downsample приводят к равнозначной эффективности (при достаточной величине выборки)

Используя разные подходы работы с дисбалансом - получаем примерно сопоставимые результаты.
На разные модели эффект от работы с дисбалансом получается раззный.

Самая лучшая наша модель, на которой проведем итоговое тестирование:

In [36]:
model = DecisionTreeClassifier(class_weight=None, max_depth=8, random_state=12345).fit(features_downsampled, target_downsampled)
f1 = f1_score(target_valid, model.predict(features_valid))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid, average=None)
print('Лучший f1_score: ', f1, 'best_auc_roc', auc_roc)

Лучший f1_score:  0.6245919477693145 best_auc_roc 0.8150887860276831


<a id='shag_4'></a>

# 4. Тестирование модели

<div class="alert alert-info"> Убрал баланс теста </div>

In [37]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.32)

best_score = 0
best_depth = 0
best_auc_roc = 0
for depth in range(1, 16, 1):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345).fit(features_downsampled, target_downsampled)
    f1 = f1_score(target_test, model.predict(features_test))
    probabilities_valid = model.predict_proba(features_test)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_test, probabilities_one_valid, average=None)
    if f1 > best_score:
        best_score = f1
        best_depth = depth
        best_auc_roc = auc_roc
print('Лучший f1_score: ', best_score,'Лучшая глубина: ', best_depth, 'best_auc_roc', best_auc_roc)

Лучший f1_score:  0.5655577299412915 Лучшая глубина:  6 best_auc_roc 0.8169614969504975


Наша модель успешно показала значение F1 = 0.55 при AUC_ROC = 0.79.

************************************

In [39]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

best_score = 0
best_depth = 0
best_auc_roc = 0
for depth in range(1, 16, 1):
    model = RandomForestClassifier(max_depth=depth, n_estimators=100, random_state=12345).fit(features_upsampled, target_upsampled )
    f1 = f1_score(target_test, model.predict(features_test))
    probabilities_valid = model.predict_proba(features_test)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_test, probabilities_one_valid, average=None)
    if f1 > best_score:
        best_score = f1
        best_depth = depth
        best_auc_roc = auc_roc
print('Лучший f1_score: ', best_score,'Лучшая глубина: ', best_depth, 'best_auc_roc', best_auc_roc)

Лучший f1_score:  0.6047745358090186 Лучшая глубина:  15 best_auc_roc 0.8458693824220855


In [41]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.4)

best_score = 0
best_depth = 0
best_auc_roc = 0
for depth in range(1, 16, 1):
    model = RandomForestClassifier(max_depth=depth, n_estimators=100, random_state=12345).fit(features_downsampled, target_downsampled)
    f1 = f1_score(target_test, model.predict(features_test))
    probabilities_valid = model.predict_proba(features_test)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_test, probabilities_one_valid, average=None)
    if f1 > best_score:
        best_score = f1
        best_depth = depth
        best_auc_roc = auc_roc
print('Лучший f1_score: ', best_score,'Лучшая глубина: ', best_depth, 'best_auc_roc', best_auc_roc)

Лучший f1_score:  0.5926860025220682 Лучшая глубина:  8 best_auc_roc 0.8482640009282821


************************************

<div class="alert alert-info"> Получили, что модели, которые были менее эффективны при сбалансированной выборке, могут показать лучший результат на несбалансированной выборке </div>

In [ ]:
def barplot(x_data, y_data, x_label="", y_label="", title=""):
    _, ax = plt.subplots(figsize=(20,3))
    #ax.set_figwidth(15)
    ax.bar(x_data, y_data, color = '#539caf', align = 'center')
    ax.errorbar(x_data, y_data, color = '#297083', ls = 'none', lw = 2, capthick = 2)
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    ax.set_title(title)

In [ ]:
importances = model.feature_importances_
factors=data.columns
df = pd.DataFrame(importances)
df['factors']=factors
df.columns = ['values', 'factors']
display(df)
barplot(factors, importances, x_label="", y_label="", title="")


<div class="alert alert-info"> Дополнив исследование графиков важности факторов, видим что самыми важными факторами являются Geography, CreditScore, Age, Tenure, HasCrCard. </div>

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*